# FargoCPT Simulation movies

This notebook shows how to make movies from output data using `matplotlib` and `ffmpeg`

First we create a new directory and change to it.

In [1]:
example_name = "500_Movie"
example_dir = f"example_dirs/{example_name}"
import os
repo_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
if not os.path.basename(os.getcwd()) == example_name:
    !mkdir -p $example_dir
    os.chdir(example_dir)
print(f"Current working directory: {os.getcwd()}")
print(f"Repository root directory: {repo_root}")

Current working directory: /home/rometsch/repo/fargocpt/examples/example_dirs/500_Movie
Repository root directory: /home/rometsch/repo/fargocpt


## Preparing a setup file

We'll take the example setup file from the examples directory and modify it in python.
If you want to create setup files for a parameter study, just copy the code and make your own setup creator script.

In [2]:
configfile = "setup.yml"
!cp $repo_root/examples/config.yml $configfile

We'll use the `ruamel.yaml` package to read and write the setup file. This can be set up to preserve comments which is very useful if you want to trace your decisions later on.

In [3]:
import ruamel.yaml
yaml = ruamel.yaml.YAML()
with open(configfile, "r") as infile:
    config = yaml.load(infile)

In [4]:
config["nbody"][1]["accretion efficiency"] = "2"
config["MonitorTimestep"] = 0.0314 # monitor scalar files around every half orbit
config["Nmonitor"] = 5 # write a snapshot after every 20 monitor timesteps = every orbit
config["Nsnapshots"] = 100
config["cps"] = 4 # use medium resolution, this is only for show, but it should still look nice
config["Frame"] = "F"
config["OmegaFrame"] = 0
config["nbody"][1]["ramp-up time"] = 10
config["nbody"][1]["accretion efficiency"] = 10
config["DiskFeedback"] = "yes"


with open(configfile, "w") as outfile:
    yaml.dump(config, outfile)

## Running the code

In [5]:
from fargocpt import run
np = 2 # Number of mpi processes. Should be equal to the number of numa nodes on your machine, check your cluster docu or run `lscpu` or `./run_fargo --print-numa` if you're on linux.
nt = 8 # Number of threads per mpi process, set it to the number of cores you want to use / number of MPI processes
run(["start", configfile], np=np, nt=nt, exe=repo_root+"/bin/fargocpt_exe", detach=False)

Running command: mpirun -np 2 --report-pid /tmp/tmpu58irt78 --map-by ppr:1:numa --bind-to numa -x OMP_WAIT_POLICY=active -x OMP_PROC_BIND=close -x OMP_PLACES=cores -x OMP_NUM_THREADS=8 /home/rometsch/repo/fargocpt/bin/fargocpt_exe start setup.yml
fargo process pid 179663

[0] MPI rank #  0 runs as process 179666
[1] MPI rank #  1 runs as process 179667
[0] MPI rank #  0 OpenMP thread #  0 of  8 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  0 of  8 on cpt-kamino
[0] MPI rank #  0 OpenMP thread #  2 of  8 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  3 of  8 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  2 of  8 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  1 of  8 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  7 of  8 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  5 of  8 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  4 of  8 on cpt-kamino
[0] MPI rank #  0 OpenMP thread #  5 of  8 on cpt-kamino
[0] MPI rank #  0 OpenMP thread #  7 of  8 on cpt-kamino
[0] MPI rank #  0 

0

## Plotting

First, we define a function to plot the top down view on the disk.

Then, we plot all images seperately and store them in a directory.

Finally, we combine the single images into a movie by leveraging ffmpeg.

In [6]:
import numpy as np
import matplotlib.colors as mplcolors
import matplotlib.pyplot as plt

def plot_field(loader, name, N, ax=None, dataunit=None, vmin=None, vmax=None, cmap="viridis", title=None):
    R, PHI, vals = loader.gas.vars2D.get(name, N, grid_for_plot=True)
    if dataunit is None:
        dataunit = vals.unit
    C = vals.to_value(dataunit)

    X = R*np.cos(PHI)
    Y = R*np.sin(PHI)

    if ax is None:
        fig, ax = plt.subplots(dpi=150)
    else:
        fig = ax.get_figure()

    norm = mplcolors.Normalize(vmin=vmin, vmax=vmax)

    # Hacky way to support arrays that are defined on the radial interfaces
    if C.shape[0] == X.shape[0]:
        C = C[:-1,:]

    pcm = ax.pcolormesh(X,Y,C, norm=norm, cmap=cmap)
    ax.set_aspect("equal")

    t = loader.snapshot_time[N].to_value("kyr")
    if title is None:
        title = ""
    else:
        title += "\n"
    title += f" t={t:.2e}kyr, N={N}"
    ax.set_title(title)

    cbar = fig.colorbar(pcm, ax=ax)
    cbar.set_label(f"{name} [{dataunit}]")
    
    return fig

In [7]:
from fargocpt import Loader
l = Loader("output/out/")

In [8]:
folder = "imgs"
!mkdir -p $folder

In [9]:
# have a progress bar
!pip install tqdm

We plot all snapshots, this might take a while.

In [10]:
from tqdm import tqdm
for n in tqdm(l.snapshots):
    fig = plot_field(l, "Sigma", n, dataunit="g/cm2", vmin=0, vmax=600, cmap="magma", title="Sigma");
    fig.savefig(f"imgs/Sigma{n}.jpg", dpi=300)
    plt.close(fig)

  0%|          | 0/101 [00:00<?, ?it/s]

100%|██████████| 101/101 [00:15<00:00,  6.61it/s]


Finally, we make the movie.

To change the file size, you can adjust framerate and the quality parameter.
Try a few settings with a small number of frames to find your sweet stop.

In [11]:
framerate = 60
quality = 20
!ffmpeg -y -framerate $framerate -i $folder/Sigma%d.jpg -c:v libx264 -crf $quality output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## The result

If you have the IPython package installed, the following should show you the video.

In [12]:
from IPython.display import Video

# Display the video
Video(os.getcwd() + "/output.mp4")